# Converting to Parquet

In [1]:
#

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1686375930022_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package('boto3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Extract the `json` files

In [3]:
import boto3
import tarfile

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def extract_json(day):
    """
    Extract the json files from the source bucket to the destination bucket.
    """
    global s3
    
    bucket_name = 'ai2-semanticscholar-cord-19'
    dst_bucket = 'bdcc2023-lab2-jwong'
    file_key = f'2020-12-0{day}/document_parses.tar.gz'
    destination_folder = 'e-15ZHHQNC8TSKL9JVQ7ZZ6OU54/2020-12/'
    tmp = '/tmp/file.tar.gz'

    s3.download_file(bucket_name, file_key, tmp)
    print('Downloaded. . .')
    with tarfile.open(tmp, 'r:gz') as tar:
        tar.extractall()
        print('Extracted. . .')
        print('Uploading. . .')
        for extracted_file in tar.getnames():
            s3.upload_file(
                Filename=f'{extracted_file}',
                Bucket=dst_bucket,
                Key=f"{destination_folder}{day}/{extracted_file}"
            )
        print('Uploaded. . .')
    os.remove(tmp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Set up the S3 client
s3 = boto3.client('s3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
extract_json(1)

In [ ]:
extract_json(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
extract_json(4)

In [ ]:
extract_json(5)

In [ ]:
extract_json(6)

## Extract using binary

In [ ]:
# Read the .tar.gz file using SparkContext
tar_path = 's3://ai2-semanticscholar-cord-19/2020-12-01/document_parses.tar.gz'

files_rdd = sc.wholeTextFiles(tar_path)

# Extract the JSON contents from the RDD
json_rdd = files_rdd.flatMap(lambda x: x[1].split('\n'))

# Convert RDD to DataFrame
df = spark.read.json(json_rdd)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import tarfile
from io import BytesIO
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the .tar.gz file as a binary stream
tar_stream = open('s3://ai2-semanticscholar-cord-19/2020-12-01/document_parses.tar.gz', 'rb')

# Extract JSON file paths from the .tar.gz file
file_paths = []
# with tarfile.open(fileobj=tar_stream, mode='r:gz') as tar:
#     for member in tar.getmembers():
#         if member.isfile() and (member.name.endswith('.json') or member.name.endswith('.jsonl')):
#             file_paths.append(member.name)

# # Read the JSON files using Spark
# json_rdd = spark.sparkContext.binaryFiles('path/to/your/file.tar.gz') \
#     .flatMap(lambda x: [(file_path, BytesIO(x[1].value).read().decode('utf-8'))
#                         for file_path in file_paths])

# # Convert RDD to DataFrame
# df = spark.read.json(json_rdd.values())

# # Show the loaded data
# df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[Errno 2] No such file or directory: 's3://ai2-semanticscholar-cord-19/2020-12-01/document_parses.tar.gz'
Traceback (most recent call last):
FileNotFoundError: [Errno 2] No such file or directory: 's3://ai2-semanticscholar-cord-19/2020-12-01/document_parses.tar.gz'



In [ ]:
# Show the loaded data
df.show()

In [10]:

# Set the path to your .tar.gz file
file_path = 's3://ai2-semanticscholar-cord-19/2020-12-01/document_parses.tar.gz'

# Create a DataFrame with the file paths and contents
df = spark.read.text(file_path)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Filter the DataFrame to include only the JSON files
json_df = df.filter(df.value.endswith('.json') | df.value.endswith('.jsonl'))

# Load the JSON files into a DataFrame
json_data = spark.read.json(json_df.value)

# Show the loaded data
json_data.show()

## Get counts

In [ ]:
from io import BytesIO

# Set the S3 bucket and file path
bucket_name = 'ai2-semanticscholar-cord-19'

# Read the .tar.gz file as a binary stream from S3
s3_client = boto3.client('s3')

for i in [3, 4, 5, 6]:
    file_key = f'2020-12-0{i}/document_parses.tar.gz'
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    tar_stream = BytesIO(response['Body'].read())

    # Extract JSON file paths from the .tar.gz file
    with tarfile.open(fileobj=tar_stream, mode='r:gz') as tar:
        print(len(tar.getmembers()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

238021
238082
239408
239278

In [15]:
counts_dict = {
    1: 236687,
    3: 238021,
    4: 238082,
    5: 239408,
    6: 239278
}

total_count = sum([val for val in counts_dict.values()])
total_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1191476

In [7]:
# Getting count of objects under a prefix in S3

# Create a Boto3 S3 client
s3_client = boto3.client('s3')

# Specify the bucket and prefix
bucket_name = 'bdcc2023-lab2-jwong'
prefix = 'e-15ZHHQNC8TSKL9JVQ7ZZ6OU54/2020-12-01/document_parses/pmc_json/'

# Call the list_objects_v2 API to retrieve the objects under the prefix
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Get the count of objects
object_count = response['KeyCount']
object_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

## Read json from temp